### 신경망(torch.nn) Model (MNIST)

- https://tutorials.pytorch.kr/beginner/basics/buildmodel_tutorial.html



In [22]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = 'cpu'

#### 신경망 정의

In [23]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            torch.nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            torch.nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 64),
            torch.nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

#### 데이터셋, 데이터로더

In [24]:
training_data = datasets.FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

test_data = datasets.FashionMNIST(
    root="./data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

#### 모델 생성

In [25]:
model = NeuralNetwork().to(device)
model.to(device)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=64, bias=True)
    (7): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=64, out_features=10, bias=True)
  )
)

#### 하이퍼파라미터 설정

In [26]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

#### 손실함수, 옵티마이저

In [27]:
loss_fn = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#### 훈련 및 테스트 함수 정의

In [28]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        X = X.to(device, dtype=torch.float)
        y = y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device, dtype=torch.float)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

#### 반복학습

In [29]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.376289  [    0/60000]
loss: 0.598204  [ 6400/60000]
loss: 0.397293  [12800/60000]
loss: 0.531277  [19200/60000]
loss: 0.447100  [25600/60000]
loss: 0.412462  [32000/60000]
loss: 0.327980  [38400/60000]
loss: 0.471803  [44800/60000]
loss: 0.409122  [51200/60000]
loss: 0.441296  [57600/60000]
Test Error: 
 Accuracy: 85.1%, Avg loss: 0.406474 

Epoch 2
-------------------------------
loss: 0.365379  [    0/60000]
loss: 0.337428  [ 6400/60000]
loss: 0.260950  [12800/60000]
loss: 0.333050  [19200/60000]
loss: 0.341817  [25600/60000]
loss: 0.313961  [32000/60000]
loss: 0.255249  [38400/60000]
loss: 0.379450  [44800/60000]
loss: 0.313267  [51200/60000]
loss: 0.387600  [57600/60000]
Test Error: 
 Accuracy: 86.3%, Avg loss: 0.373725 

Epoch 3
-------------------------------
loss: 0.259013  [    0/60000]
loss: 0.298778  [ 6400/60000]
loss: 0.214416  [12800/60000]
loss: 0.272891  [19200/60000]
loss: 0.289748  [25600/60000]
loss: 0.270691  [32000/600

#### 확인

In [31]:
import matplotlib.pyplot as plt
import numpy as np

label_tags = {
    0: 'T-Shirt', 
    1: 'Trouser', 
    2: 'Pullover', 
    3: 'Dress', 
    4: 'Coat', 
    5: 'Sandal', 
    6: 'Shirt',
    7: 'Sneaker', 
    8: 'Bag', 
    9: 'Ankle Boot'
}

columns = 6
rows = 6

fig = plt.figure(figsize=(10,10))

for i in range(1, columns*rows+1):
    data_idx = np.random.randint(len(test_dataloader))
    input_img = test_data[data_idx][0].unsqueeze(dim=0)

    output = model(input_img)
    _, argmax = torch.max(output, 1)
    pred = label_tags[argmax.item()]
    label = label_tags[test_data[data_idx][1]]

    fig.add_subplot(rows, columns, i)
    if pred == label:
        plt.title(pred + ', right !')
        cmap = 'Blues'
    else:
        plt.title('Not ' + pred + ' but ' +  label)
        cmap = 'Reds'
    plot_img = test_data[data_idx][0][0,:,:]
    plt.imshow(plot_img, cmap=cmap)
    plt.axis('off')

plt.show() 

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 512])

<Figure size 1000x1000 with 0 Axes>